In [15]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Load the client configuration from JSON
CLIENT_CONFIG = {"installed":{"client_id":"342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com","project_id":"ambient-isotope-384119","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}


# Set up the credentials object using the client configuration
creds = None
flow = InstalledAppFlow.from_client_config(CLIENT_CONFIG, scopes=['https://www.googleapis.com/auth/calendar.readonly'])
creds = flow.run_local_server(port=0)

# Build the Google Calendar API client
service = build("calendar", "v3", credentials=creds)



Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51440%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=5xW4TYBz03i5NoJmGrTK4uilgrojiN&access_type=offline


HttpError: <HttpError 400 when requesting https://www.googleapis.com/calendar/v3/calendars/primary/events?timeMin=2014-01-01T00%3A00%3A00T00%3A00%3A00.000000Z&timeMax=2014-01-02T00%3A00%3A00T00%3A00%3A00.000000Z&singleEvents=true&orderBy=startTime&timeZone=Asia%2FKarachi&alt=json returned "Bad Request". Details: "[{'domain': 'global', 'reason': 'badRequest', 'message': 'Bad Request'}]">

In [ ]:
import concurrent.futures
import datetime

# Set the time zone to use for the query
timezone = pytz.timezone('Asia/Karachi')
calendar_id = 'en.pk.official#holiday@group.v.calendar.google.com'

# Set the start and end dates for the query
start_date = datetime.datetime(2014, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

# Define a function to process a date range
def process_date_range(start_date, end_date):
    # Loop through each day in the date range and query the events for that day
    current_date = start_date
    while current_date < end_date:
        # Set the start and end of day times for the current date in the specified time zone
        start_of_day = timezone.localize(current_date).isoformat()
        end_of_day = (timezone.localize(current_date) + datetime.timedelta(days=1)).isoformat()

        # Call the events().list() method with additional parameters to get events in the specified time range and location
        try:
            calendar_events = service.events().list(calendarId=calendar_id, timeMin=start_of_day, timeMax=end_of_day, singleEvents=True, orderBy='startTime', timeZone='pakistan').execute()
            print(current_date.strftime('%Y-%m-%d'))
            for event in calendar_events['items']:
                print(event['summary'])
                    # Print the event details

        except HttpError as error:
            print(f'An error occurred: {error}')

        # Move to the next day
        current_date += datetime.timedelta(days=1)

# Divide the date range into smaller chunks
delta = datetime.timedelta(days=90)
date_ranges = [(start_date + i * delta, min(start_date + (i + 1) * delta, end_date)) for i in range((end_date - start_date) // delta + 1)]

# Process the date ranges concurrently using multiple threads
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_date_range, start, end) for (start, end) in date_ranges]

    # Wait for all the threads to complete
    for future in concurrent.futures.as_completed(futures):
        try:
            future.result()
        except Exception as e:
            print(f'An error occurred: {e}')


2014-01-01
An error occurred: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:2633)
An error occurred: [SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:2633)
2015-12-22
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
2016-06-19
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
2016-12-16
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
An error occurred: [SSL: DECRYPTION_FAILED_OR_BAD_RECORD_MAC] decryption failed or bad record mac (_ssl.c:2633)
2017-09-12
2017-06-14


In [4]:
import threading
import datetime
import pytz
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


# Set the time zone to use for the query
timezone = pytz.timezone('Asia/Karachi')
calendar_id = 'en.pk.official#holiday@group.v.calendar.google.com'

# Set the start and end dates for the query
start_date = datetime.datetime(2014, 1, 1)
end_date = datetime.datetime(2023, 1, 1)

# Set the number of threads to use
num_threads = 10

# Create a list of date ranges to be queried by each thread
date_ranges = []
delta = (end_date - start_date) / num_threads
for i in range(num_threads):
    start = start_date + i * delta
    end = start_date + (i+1) * delta
    date_ranges.append((start, end))

# Define a function to query events for a given date range
def query_events(start_date, end_date):
    try:
        # Build the service object using credentials
        CLIENT_CONFIG = {"installed":{"client_id":"342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com","project_id":"ambient-isotope-384119","auth_uri":"https://accounts.google.com/o/oauth2/auth","token_uri":"https://oauth2.googleapis.com/token","auth_provider_x509_cert_url":"https://www.googleapis.com/oauth2/v1/certs","client_secret":"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","redirect_uris":["http://localhost"]}}


        # Set up the credentials object using the client configuration
        creds = None
        flow = InstalledAppFlow.from_client_config(CLIENT_CONFIG, scopes=['https://www.googleapis.com/auth/calendar.readonly'])
        creds = flow.run_local_server(port=0)

        # Build the Google Calendar API client
        service = build("calendar", "v3", credentials=creds)


        # Loop through each day in the date range and query the events for that day
        current_date = start_date
        while current_date < end_date:
            # Set the start and end of day times for the current date in the specified time zone
            start_of_day = timezone.localize(current_date).isoformat()
            end_of_day = (timezone.localize(current_date) + datetime.timedelta(days=1)).isoformat()

            # Call the events().list() method with additional parameters to get events in the specified time range and location
            calendar_events = service.events().list(calendarId=calendar_id, timeMin=start_of_day, timeMax=end_of_day, singleEvents=True, orderBy='startTime', timeZone='pakistan').execute()

            # Print the events for the current day
            print(current_date.strftime('%Y-%m-%d'))
            for event in calendar_events['items']:
                print(event['summary'])

            # Move to the next day
            current_date += datetime.timedelta(days=1)

    except HttpError as error:
        print(f'An error occurred: {error}')

# Create a list of threads and start each thread with a different date range
threads = []
for start, end in date_ranges:
    t = threading.Thread(target=query_events, args=(start, end))
    threads.append(t)
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51748%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=ZZtinNY2qTR1uBrHzInzf8zaH3gzcb&access_type=offlinePlease visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51745%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar.readonly&state=t6FK25nIzMZpcb1FMgbjak8TW0hiWU&access_type=offlinePlease visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=342101447417-fdd4oitdd57qq41d4hipdko946skvk8u.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A51744%2F&scope=https%3A%2F%2Fwww.

2014-12-14
2016-12-09
2017-10-14
2019-09-12
2018-07-24
2015-11-06
2020-05-11
2021-04-03
2022-03-03
2014-01-25
2014-12-15
2016-12-10
2015-11-07
2019-09-13
2018-07-252017-10-15

2021-04-04
2020-05-12
2022-03-04
2014-01-26
2016-12-11
2014-12-16
2015-11-08
2019-09-14
2017-10-16
2018-07-26
2020-05-13
2021-04-05
2022-03-05
2014-01-27
2016-12-12
2014-12-17
2019-09-15
2015-11-09
2018-07-27
2021-04-06
2017-10-17
2020-05-14
2014-01-28
2022-03-06
2014-12-18
2015-11-10
2016-12-13
2019-09-16
2021-04-07
2018-07-28
2017-10-18
2020-05-15
2014-01-29
2022-03-07
2015-11-11
2014-12-19
2016-12-14
2019-09-17
2021-04-08
2018-07-29
2020-05-16
2017-10-19
2014-01-30
2014-12-20
2022-03-08
2015-11-12
2016-12-15
2019-09-18
2018-07-30
2021-04-09
2020-05-17
2017-10-20
2022-03-09
2014-12-21
2014-01-31
2015-11-13
2016-12-16
2019-09-19
2018-07-31
2021-04-10
2020-05-18
2022-03-10
2017-10-21
2014-02-01
2014-12-22
2015-11-14
2016-12-17
2018-08-01
2019-09-20
2021-04-11
2020-05-19
2022-03-11
2017-10-22
2015-11-15
2016-12-18

2015-02-25
2019-11-22
2014-04-07
2021-06-16
2022-05-15
2018-10-05
2020-07-22
2017-02-20
2017-12-26
2016-01-19
2015-02-26
2014-04-08
2021-06-17
2019-11-23
2020-07-23
2018-10-06
2022-05-16
2017-02-21
2017-12-27
2016-01-20
2015-02-27
2021-06-18
2019-11-24
2014-04-09
2020-07-24
2018-10-07
2022-05-17
2017-02-22
2017-12-28
2016-01-21
2015-02-28
2021-06-19
2019-11-25
2014-04-10
2020-07-25
2017-02-23
2022-05-18
2017-12-29
2016-01-22
2018-10-08
2021-06-20
2019-11-26
2015-03-01
2020-07-26
2017-02-24
2014-04-11
2016-01-23
2022-05-19
2018-10-09
2017-12-30
2015-03-02
2021-06-21
2019-11-27
2020-07-27
2014-04-12
2017-02-25
2016-01-24
2018-10-10
2017-12-31
2022-05-20
2021-06-22
2019-11-28
2015-03-03
2020-07-28
2014-04-13
2016-01-25
2017-02-26
2018-01-01
2018-10-11
2022-05-21
2015-03-04
2019-11-29
2021-06-23
2020-07-29
2016-01-26
2014-04-14
2017-02-27
2018-01-02
2018-10-12
2022-05-22
2015-03-05
2019-11-30
2021-06-24
2018-01-03
2016-01-27
2020-07-30
2014-04-15
2017-02-28
2018-10-13
2022-05-23
2015-03-06

2020-10-03
2020-02-03
2016-03-30
2022-07-26
2014-06-19
2017-05-04
2015-05-09
2021-08-29
2020-02-04
2018-03-10
2018-12-18
2016-03-31
2022-07-27
2020-10-04
2014-06-20
2017-05-05
2015-05-10
2021-08-30
2018-12-19
2020-10-05
2022-07-28
2020-02-05
2018-03-11
2014-06-21
2016-04-01
2017-05-06
2015-05-11
2021-08-31
2020-10-06
2022-07-29
2018-12-20
2018-03-12
2020-02-06
2014-06-22
2016-04-02
2015-05-12
2017-05-07
2021-09-01
2020-10-07
2018-12-21
2022-07-30
2020-02-07
2014-06-23
2016-04-03
2018-03-13
2015-05-13
2017-05-08
2021-09-02
2020-10-08
2018-12-22
2014-06-24
2022-07-31
2016-04-04
2017-05-09
2020-02-08
2015-05-14
2018-03-14
2021-09-03
2020-10-09
2018-12-23
2022-08-01
2014-06-252016-04-05

2020-02-09
2017-05-10
2021-09-04
2015-05-15
2018-03-15
2020-10-10
2018-12-24
2014-06-26
2020-02-10
2017-05-11
2016-04-06
2015-05-16
2018-03-16
2022-08-02
2021-09-05
2020-10-11
2018-12-25
2020-02-11
2014-06-27
2015-05-17
2017-05-12
2016-04-07
2022-08-03
2018-03-17
2020-10-12
2021-09-06
2018-12-26
2015-05-18

2018-05-21
2020-04-17
2016-06-12
2015-07-21
2019-03-02
2014-09-02
2017-07-16
2020-12-16
2022-10-09
Eid Milad un-Nabi
2021-11-11
2018-05-22
2020-04-18
2016-06-13
2015-07-22
2019-03-03
2014-09-03
2017-07-17
2020-12-17
2022-10-10
2021-11-12
2020-04-19
2018-05-23
2016-06-14
2015-07-23
2019-03-04
2014-09-04
2017-07-18
2020-12-18
2021-11-13
2018-05-24
2022-10-11
2015-07-24
2016-06-15
2019-03-05
2014-09-05
2017-07-19
2020-12-19
2021-11-14
2018-05-25
2022-10-12
2016-06-16
2015-07-25
2019-03-06
2014-09-06
2017-07-20
2020-12-20
2022-10-13
2018-05-26
2021-11-15
2016-06-17
2015-07-26
2020-12-21
2019-03-07
2014-09-07
2017-07-21
2018-05-27
2022-10-14
2021-11-16
2016-06-18
2015-07-27
2019-03-08
2020-12-22
2014-09-08
2018-05-28
2017-07-22
2022-10-15
2021-11-17
2019-03-09
2016-06-19
2020-12-23
2015-07-28
2014-09-09
2022-10-16
2017-07-23
2018-05-29
2021-11-18
2019-03-10
2015-07-29
2016-06-20
2014-09-10
2020-12-24
2022-10-17
2017-07-24
2021-11-19
2018-05-30
2019-03-11
2016-06-21
2014-09-11
2020-12-25
201

In [ ]:
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [ ]:
scopes = ['https://www.googleapis.com/auth/calendar']


In [ ]:
flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)


In [ ]:
credentials = flow.run_console()
